
 RoBERTa model using Transformers

In [ ]:
# Install the transformers library
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00


## Import Required modules

In [1]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers import pipeline

AutoTokenizer, AutoModelForSequenceClassification, and Trainer from transformers: These are key components from the Hugging Face Transformers library for loading pre-trained tokenizers and models, and for training or evaluating models.
pipeline from transformers: A high-level abstraction in the Transformers library that simplifies using pre-trained models for various tasks, including sentiment analysis.

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Approach-1 Using pipeline

In [3]:
sentiment_task = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
sentiment = sentiment_task("I watched Oppenheimer movie, one the finest and incredible movie")
sentiment

[{'label': 'POSITIVE', 'score': 0.9998806715011597}]

In [5]:
sentiment_neg = sentiment_task("This movie is worst and terrible")
sentiment_neg

[{'label': 'NEGATIVE', 'score': 0.9998193383216858}]

## Tokenization and Model initialization

In [13]:
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

This code initializes a pre-trained RoBERTa model for sentiment analysis:

model_name = "siebert/sentiment-roberta-large-english": This line defines a variable model_name and assigns it the string identifier for a specific pre-trained RoBERTa model from the Hugging Face model hub. This particular model is fine-tuned for sentiment analysis in English.
tokenizer = AutoTokenizer.from_pretrained(model_name): This line loads the tokenizer associated with the specified pre-trained model. The tokenizer is responsible for converting raw text into a format that the model can understand (e.g., numerical IDs). AutoTokenizer.from_pretrained() automatically downloads and loads the correct tokenizer for the given model_name.
model = AutoModelForSequenceClassification.from_pretrained(model_name): This line loads the pre-trained RoBERTa model itself. AutoModelForSequenceClassification.from_pretrained() downloads and loads the model architecture and its pre-trained weights. This model is specifically designed for sequence classification tasks like sentiment analysis, where the goal is to classify an entire input sequence (like a tweet) into a specific category (e.g., positive or negative sentiment).

## Dataset preparation

- Text to Tokens
- Tokens to Unique IDS
- Map Tokens with its unique IDs in a dictionary


This code defines a custom dataset class called SimpleDataset that is used to prepare your tokenized text data for training or evaluation with a Hugging Face Transformer model.

Here's a breakdown:

class SimpleDataset:: This line defines a new class named SimpleDataset.
def __init__(self, tokenized_texts):: This is the constructor of the class. It takes one argument, tokenized_texts, which is expected to be a dictionary containing the tokenized input data (like the output from a Hugging Face tokenizer). It stores this data in the instance variable self.tokenized_texts.
def __len__(self):: This method is required for a dataset class. It defines how to get the size of the dataset. In this case, it returns the number of sequences in the dataset by checking the length of the "input_ids" list within the tokenized_texts dictionary.
def __getitem__(self, idx):: This method is also required for a dataset class. It defines how to retrieve a single item from the dataset given an index (idx). It returns a dictionary where the keys are the same as in the tokenized_texts dictionary (e.g., "input_ids", "attention_mask") and the values are the corresponding elements at the given index idx. This allows iterating through the dataset and getting individual tokenized examples.
In essence, this SimpleDataset class acts as a wrapper around your tokenized data, making it compatible with the Trainer class from the Hugging Face Transformers library for model training and evaluation.


In [19]:
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

## Approach -2 Sample User Text- Using pre-trained models

In [24]:
tweets = [
    "Just had the best meal ever at my favorite restaurant! The food was amazing, and the service was top-notch. Feeling so satisfied and happy right now! 😃🍔🥗 #Foodie #HappyCustomer",
    "Feeling really frustrated after sitting in traffic for hours today. Missed an important meeting and wasted so much time. Not a great start to the day. 😤🚗 #TrafficWoes #Stressed"
]

## Tokenize the user input text

In [25]:
tokenized_texts = tokenizer(tweets,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

## Create Model Trainer

In [26]:
trainer = Trainer(model=model)

In [27]:
predictions = trainer.predict(pred_dataset) #run predictions

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ebabar (ebabar-fast-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Model Evaluation- Predictions

In [28]:
preds = predictions.predictions.argmax(-1) #get prediction labels
labels = pd.Series(preds).map(model.config.id2label)  #convert prediction into POSITIVE or NEGATIVE
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1) # ACCURACY SCORE

In [29]:
scores

array([0.9989399 , 0.99949604], dtype=float32)

In [30]:
labels

,0
0,POSITIVE
1,NEGATIVE


## RoBERTa Sentiment results on user input

In [31]:
df = pd.DataFrame(list(zip(tweets,preds,labels,scores)), columns=['text','pred','label','score'])
df.head()

,text,pred,label,score
0,Just had the best meal ever at my favorite res...,1,POSITIVE,0.998940
1,Feeling really frustrated after sitting in tra...,0,NEGATIVE,0.999496


## Approach-3 On Custom Dataset

Download the dataset and upload the dataset in the Google Colab to proceed further

Dataset: [Twitter Sentiment Analysis Dataset](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset)

In [ ]:
data = pd.read_csv("Tweets.csv")
data.tail()

,textID,text,selected_text,sentiment
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive
27480,6f7127d9d7,All this flirting going on - The ATG smiles...,All this flirting going on - The ATG smiles. Y...,neutral


In [ ]:
data_tweets = data['text'].dropna().astype('str').tolist()

In [ ]:
tokenized_data = tokenizer(data_tweets,truncation=True,padding=True)
pred_data = SimpleDataset(tokenized_data)

In [ ]:
evaluate = trainer.predict(pred_data) #will take some time

In [ ]:
new_preds = evaluate.predictions.argmax(-1) #get prediction labels
new_labels = pd.Series(new_preds).map(model.config.id2label) #convert prediction into POSITIVE or NEGATIVE
new_scores = (np.exp(evaluate[0])/np.exp(evaluate[0]).sum(-1,keepdims=True)).max(1) # ACCURACY SCORE

In [ ]:
final_df = pd.DataFrame(list(zip(data_tweets,new_preds,new_labels,new_scores)), columns=['tweets','target','sentiment','accuracy'])
final_df.head(10)

,tweets,target,sentiment,accuracy
0,"I`d have responded, if I were going",0,NEGATIVE,0.994793
1,Sooo SAD I will miss you here in San Diego!!!,0,NEGATIVE,0.995364
2,my boss is bullying me...,0,NEGATIVE,0.999434
3,what interview! leave me alone,0,NEGATIVE,0.997404
4,"Sons of ****, why couldn`t they put them on t...",0,NEGATIVE,0.998303
5,http://www.dothebouncy.com/smf - some shameles...,1,POSITIVE,0.994551
6,2am feedings for the baby are fun when he is a...,1,POSITIVE,0.997768
7,Soooo high,1,POSITIVE,0.997417
8,Both of you,0,NEGATIVE,0.829015
9,Journey!? Wow... u just became cooler. hehe....,1,POSITIVE,0.997250
